In [1]:
import math
from collections import defaultdict
from collections import Counter
import pandas as pd

In [184]:

import math
from collections import defaultdict
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics import ndcg_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import gzip
import random

class Index:
    def __init__(self) -> None:
        self.index = defaultdict(dict)
        self.embeddings = None
        self.doc_lengths = {}
        self.avgdl = 0
        self.idf = {}
        self.cf = {}

    @staticmethod
    def preprocess_text(text):
        stop_words = set(stopwords.words('english'))
        ps = PorterStemmer()
        text = text.lower()
        tokens = word_tokenize(text)
        tokens = [ps.stem(token) for token in tokens if token.isalnum() and token not in stop_words]
        return tokens

    def load_file(self, file_name):
        file_extension = file_name.split('.')[-1].lower()

        if file_extension == 'csv':
            self.docs = pd.read_csv(file_name)
        elif file_extension == 'tsv':
            self.docs = pd.read_csv(file_name, delimiter='\t',header=None)
            self.docs.columns = ['pid', 'passage']
        else:
            raise ValueError("Unsupported file format. Supported formats: CSV (.csv) and TSV (.tsv)")

    def build_index(self, file_name: str):
        self.load_file(file_name=file_name)

        self.docs['passage'] = self.docs['passage'].apply(self.preprocess_text)

        total_tokens = 0
        for index, row in self.docs.iterrows():
            doc_id, tokens = row['pid'], row['passage']
            total_tokens += len(tokens)
            for term in tokens:
                self.index[term][doc_id] = self.index[term].get(doc_id, 0) + 1
                if term not in self.cf:
                    self.cf[term] = 1
                else:
                    self.cf[term] += 1
                

            self.doc_lengths[doc_id] = len(tokens)

        self.avgdl = total_tokens / len(self.docs)
        self.compute_idf()

        corpus = [' '.join(i) for i in list(self.docs['passage'])]
        self.vectorizer = TfidfVectorizer()
        self.embeddings = self.vectorizer.fit_transform(corpus)

    def compute_idf(self):
        total_docs = len(self.docs)
        for term in self.index:
            doc_freq = len(self.index[term])
            self.idf[term] = math.log((total_docs - doc_freq + 0.5) / (doc_freq + 0.5) + 1.0)

    def save_index(self, file_name: str):
        with gzip.open(file_name, 'wb', compresslevel=9) as file:
            pickle.dump({'index': self.index, 'doc_lengths': self.doc_lengths, 'avgdl': self.avgdl, 'idf': self.idf, 'cf': self.cf, 'vectorizer': self.vectorizer, "embeddings": self.embeddings}, file)

    def load_index(self, file_name: str):
        with gzip.open(file_name, 'rb') as file:
            data = pickle.load(file)
            self.index = data['index']
            self.doc_lengths = data['doc_lengths']
            self.avgdl = data['avgdl']
            self.idf = data['idf']
            self.cf = data['cf']
            self.embeddings = data['embeddings']
            self.vectorizer = data['vectorizer']


class RetrievalModel:
    def __init__(self, index: Index) -> None:
        self.index = index
        self.len_C = len(self.index.index)
    
    def preselect_docs(self, query, min_selected_docs=2):
        query_terms = set(query)

        relevant_docs = set()

        for term in query_terms:
            relevant_docs.update(self.index.index.get(term, {}).keys())

        return relevant_docs


    def query_likelihood(self, query, lambd):
        scores = {}
        
        for doc_id in self.preselect_docs(query):
            len_doc = self.index.doc_lengths[doc_id]

            p_q_Md = 0

            for term in query:
                df = self.index.index.get(term, {}).get(doc_id, 0)
                if len(self.index.index[term]) == 0: continue

                cf = self.index.cf[term]
                p_q_Md += np.log((1 - lambd) * (df / len_doc) + (lambd * (cf / self.len_C)))

            scores[doc_id] = p_q_Md

        sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        return sorted_scores
    
    def bm25_ranking(self, query, k1 = 1.2, b = 0.75):
        scores = {}

        for term in query:
            score = 0

            if len(self.index.index[term]) == 0: continue
            
            idf_value = self.index.idf[term]
            
            for doc_id, tf in self.index.index[term].items():
                len_doc = self.index.doc_lengths[doc_id]
                tf =  self.index.index.get(term, {}).get(doc_id, 0)
                score += idf_value * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * len_doc / self.index.avgdl)))

                if not doc_id in scores:
                    scores[doc_id] = score
                else:
                    scores[doc_id] += score
        
        # sort scores / ranking
        sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        return sorted_scores

    def embeddings_cosign_sim(self, query):
        query = [" ".join(list(query))]
        vec_query = self.index.vectorizer.transform(query)
        
        cos = cosine_similarity(self.index.embeddings, vec_query)

        scores = dict(zip(self.index.doc_lengths.keys(), cos.flatten()))
        
        non_zero_scores = {k: v for k, v in scores.items() if v != 0}

        sorted_scores = {k: v for k, v in sorted(non_zero_scores.items(), key=lambda item: item[1], reverse=True)}

        return sorted_scores
        
    def evaluate_model(self, qrel_file, query_file, lambd=0.5, output_file='evaluation_results.csv'):
        # Parse qrel file
        qrel_data = pd.read_csv(qrel_file)

        # Read query file and preprocess queries
        query_data = pd.read_csv(query_file)
        query_data['query'] = query_data['query'].apply(self.index.preprocess_text)

        # Create a DataFrame to store results
        results = pd.DataFrame(columns=['qid','ql_ndcg', 'bm25_ndcg', 'cos_sim_ndcg'])
        results['qid'] = query_data['qid']
        results['ql_ndcg'] = None
        results['bm25_ndcg'] = None
        results['cos_sim_ndcg'] = None

        # query_data = query_data.loc[query_data['qid'] == 722737]

        # Evaluate each query
        for qid, query in zip(query_data['qid'], query_data['query']):
            relevant_docs = qrel_data[(qrel_data['Topic'] == qid) & (qrel_data['Relevancy'] == 1)]['Document#'].tolist()

            # Query Likelihood
            ql_scores = self.query_likelihood(query, lambd)
            ranked_docs_ql = np.array(list(ql_scores.keys()))
            ranked_values_ql = np.array(list(ql_scores.values()))

            # Create binary list for relevant and non-relevant documents
            y_true_ql = np.isin(ranked_docs_ql, relevant_docs)

            # BM25
            bm25_scores = self.bm25_ranking(query)
            ranked_docs_bm25 = np.array(list(bm25_scores.keys()))
            ranked_values_bm25 = np.array(list(bm25_scores.values()))

            # Create binary list for relevant and non-relevant documents
            y_true_bm25 = np.isin(ranked_docs_bm25, relevant_docs)

            # Embeddings
            cos_sim_scores = self.embeddings_cosign_sim(query)
            ranked_docs_cos_sim = np.array(list(cos_sim_scores.keys()))
            ranked_values_cos_sim = np.array(list(cos_sim_scores.values()))

            # Create binary list for relevant and non-relevant documents
            y_true_cos_sim = np.isin(ranked_docs_cos_sim, relevant_docs)

            if len(y_true_ql) < 2 or len(y_true_bm25) < 2 or len(y_true_cos_sim) < 2: continue

            # Calculate NDCG scores
            ndcg_ql = ndcg_score([y_true_ql], [ranked_values_ql])
            ndcg_bm25 = ndcg_score([y_true_bm25], [ranked_values_bm25])
            ndcg_cos_sim = ndcg_score([y_true_cos_sim], [ranked_values_cos_sim])

            # Append results to the DataFrame
            results.loc[results['qid'] == qid] = [qid, ndcg_ql, ndcg_bm25, ndcg_cos_sim]

        # Save results to a CSV file
        results.to_csv(output_file, index=False)

In [157]:
data_fine_name = r"MSMARCO_SMALL\collection_small.csv"
# data_fine_name = r"data\collection.tsv"
index_file_name = 'index.json.gz'

index = Index()
build = False

if build:
    index.build_index(data_fine_name)
    index.save_index(index_file_name)
else:
    index.load_index(index_file_name)

In [185]:
query_file = r"MSMARCO_SMALL\queries_small.csv"
qrel_file = r"MSMARCO_SMALL\qrel_small.csv"
retrival_model = RetrievalModel(index)

retrival_model.evaluate_model(qrel_file, query_file)

In [186]:
import time

query_file = r"MSMARCO_SMALL\queries_small.csv"
qrel_file = r"MSMARCO_SMALL\qrel_small.csv"

retrival_model = RetrievalModel(index)

query_data = pd.read_csv(query_file)
query_data['query'] = query_data['query'].apply(Index.preprocess_text)

query1 = query_data['query'].iloc[0]
query = query_data.loc[query_data['qid'] == 722737, 'query'].iloc[0]

start_time = time.time()
retrival_model.query_likelihood(query, 0.35)
end_time = time.time()
print(f"query_likelihood took {end_time - start_time} seconds")

start_time = time.time()
retrival_model.bm25_ranking(query)
end_time = time.time()
print(f"bm25_ranking took {end_time - start_time} seconds")

start_time = time.time()
retrival_model.embeddings_cosign_sim(query)
end_time = time.time()
print(f"embeddings_cosign_sim took {end_time - start_time} seconds")

query_likelihood took 0.0 seconds
bm25_ranking took 0.0 seconds
embeddings_cosign_sim took 0.04980301856994629 seconds


## Test with more efficient retrieval models

In [147]:
def preselect_docs(index, query):
        query_terms = set(query)

        relevant_docs = set()

        for term in query_terms:
            relevant_docs.update(index.get(term, {}).keys())
        
        return relevant_docs


def compute_idf(index, doc_lengths):
    idf = {}

    total_docs = len(doc_lengths)
    for term in index:
        doc_freq = len(index[term])
        idf[term] = math.log((total_docs - doc_freq + 0.5) / (doc_freq + 0.5) + 1.0)
    return idf

def query_likelihood(index, doc_lengths, query, lambd):
    scores = {}
    
    for doc_id in preselect_docs(index, query):
        len_C = 12

        len_doc = doc_lengths[doc_id]
        
        p_q_Md = 0
        for term in query:
            df = index.get(term, {}).get(doc_id, 0)
            cf = sum(index[term].values())
            
            ts = (1 - lambd) * (df / len_doc) + (lambd * (cf / len_C))

            if ts != 0:
                p_q_Md += math.log(ts)

        scores[doc_id] = p_q_Md

    sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
    return sorted_scores

def query_likelihood_new(index, doc_lengths, query, lambd):
    scores = {}
    nr_terms = {}

    len_C = 12
    for term in query:
        cf = sum(index[term].values())

        for doc_id, df in index[term].items():
            
            doc_len = doc_lengths[doc_id]

            ts = (1 - lambd) * (df / doc_len) + (lambd * (cf / len_C))

            if not doc_id in scores:
                scores[doc_id] = math.log(ts)
            else:
                scores[doc_id] += math.log(ts)

            if not doc_id in nr_terms:
                nr_terms[doc_id] = 


    sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
    return sorted_scores

    N = len(inverted_index)  # Total number of documents in the collection

    doc_scores = np.zeros(len(doc_lengths))
    doc_lengths = np.array(list(doc_lengths.values()))

    for term in query:
        if term in inverted_index:
            doc_term_freqs = np.array(list(inverted_index[term].values()))


            term_prob = doc_term_freqs / doc_lengths
            collection_prob = np.sum(doc_term_freqs) / np.sum(doc_lengths)

            doc_scores = doc_scores + np.log((1 - lambda_value) * term_prob + lambda_value * collection_prob)

    return doc_scores

def bm25_ranking(index, doc_lengths, idf, query):
        # Hyperparams to specify
        k1 = 1.2
        b = 0.75
        scores = {}

        total_tokens = sum(value for inner_dict in index.values() for value in inner_dict.values())
        avgdl = total_tokens / len(doc_lengths)

        # Looping through the different docs
        for doc_id in preselect_docs(index, query):
            len_doc = doc_lengths[doc_id]
            if len_doc == 0:
                continue
            score = 0
            
            # Loop for term in query in the doc
            for term in query:
                # Calculating/updating the score
                tf =  index.get(term, {}).get(doc_id, 0)
                idf_value = idf.get(term, 0)  # Use 0 as the default value if term is not in idf
                score += idf_value * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * len_doc / avgdl)))

            scores[doc_id] = score 
        
        # sort scores / ranking
        sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        return sorted_scores

def bm25_ranking_new(index, doc_lengths, idf, query):
        # Hyperparams to specify
        k1 = 1.2
        b = 0.75
        scores = {}

        total_tokens = sum(value for inner_dict in index.values() for value in inner_dict.values())
        avgdl = total_tokens / len(doc_lengths)

        for term in query:
            score = 0
            idf_value = idf[term]
            for doc_id, tf in index[term].items():
                len_doc = doc_lengths[doc_id]
                
                score = idf_value * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * len_doc / avgdl)))

                if not doc_id in scores:
                    scores[doc_id] = score
                else:
                    scores[doc_id] += score
        
        # sort scores / ranking
        sorted_scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1], reverse=True)}
        return sorted_scores

doc_lengths = {1:2, 2:3, 3:7}

query = ["Apple", "Phone"]

index = {"Apple": {1:1, 2:2, 3:1},
         "Samsung": {1:1, 3:3},
         "Phone": {2:1, 3:3}}



idf = compute_idf(index, doc_lengths)
lambd = 0.35
print(bm25_ranking(index, doc_lengths, idf, query))
print(bm25_ranking_new(index, doc_lengths, idf, query))

print(query_likelihood(index, doc_lengths, query, lambd))
print(query_likelihood_new(index, doc_lengths, query, lambd))
# jm_smoothed_query_likelihood(index, query, doc_lengths, lambda_value=lambd)

{3: 0.7384931496694467, 2: 0.7210401540807048, 1: 0.16786803644225698}
{3: 0.7384931496694467, 2: 0.7210401540807048, 1: 0.16786803644225698}
{2: -1.6964492894237302, 3: -2.4911848197200785, 1: -2.9656342423967117}
{1: -0.8171998292299242, 2: -1.6964492894237302, 3: -2.4911848197200785}
